In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Learning Catalogue.xlsx to Learning Catalogue (1).xlsx
Saving Sample_Skills.xlsx to Sample_Skills (1).xlsx


In [3]:
!pip install -q xlrd

In [4]:
df = pd.read_excel('Learning Catalogue.xlsx')
df = pd.DataFrame(df, columns=['Description'])
stop_words_l=stopwords.words('german')
df['documents_cleaned']=df.Description.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(df.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(df.documents_cleaned)

In [5]:
pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T).toarray()
pairwise_differences=euclidean_distances(tfidf_vectors)

In [6]:
def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Description: {df.iloc[doc_id]["Description"]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Description: {df.iloc[ix]["Description"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')  

Description: Die Digitalisierung hat unser gesamtes Leben völlig neu geprägt. Sie hat viele positive Seiten, aber auch negative Folgen wie Stress und Konzentrationsstörungen. In diesem eLearning analysieren Sie die Ursachen der digitalen Belastung und erhalten Tipps, wie Sie mit den Folgen der dauerhaften Erreichbarkeit umgehen können. Außerdem werden Sie mehrere Möglichkeiten kennenlernen und ausprobieren, mit denen Sie zurück in die Entspannung finden können. Außerdem können Sie mit unserem Digital-Detox-Notfallkoffer unmittelbar digitale Belastung erkennen und bekommen Werkzeuge an die Hand, mit denen Sie direkt etwas dagegen tun können.

Lernziele
- Sie (er)kennen die Ursachen der digitalen Belastung wie Multitasking, störenden Unterbrechungen und sofortige Belohnung und können hiermit bewusst umgehen.
- Sie analysieren Ihre Nutzung von digitalen Medien und Endgeräten.
- Sie entwickeln Strategien für einen bewussteren Umgang mit digitalen Medien und Endgeräten und nutzen Digital-De

In [7]:
pip install sentence_transformers

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [9]:
sentence_embeddings = model.encode(df.documents_cleaned)

In [10]:
sentence_embeddings.shape

(120, 768)

In [11]:
sentence_embeddings

array([[-0.26230934,  0.9981962 ,  1.0188303 , ..., -0.8327339 ,
         0.38151774,  0.38453272],
       [-0.28432453,  1.0309218 ,  1.4001635 , ..., -0.3679473 ,
         0.28369394,  0.2610441 ],
       [-0.5109114 ,  1.4128888 ,  1.1264514 , ..., -0.8714148 ,
         0.42052853,  0.09516983],
       ...,
       [-0.16072486,  0.5732545 ,  0.8819661 , ..., -0.43790898,
         0.0933486 ,  0.06831039],
       [-0.18239552,  0.7952059 ,  1.4253217 , ..., -0.51708984,
         0.08842055, -0.03301304],
       [-0.42126423,  0.8632714 ,  1.2000933 , ..., -0.4977098 ,
         0.15014051,  0.2174381 ]], dtype=float32)

In [12]:
df2 = pd.read_excel('Sample_Skills.xlsx')
df2 = pd.DataFrame(df2, columns=['Skill'])
stop_words_l=stopwords.words('german')
df2['skill_cleaned']=df2.Skill.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

In [13]:
sentence_embeddings2 = model.encode(df2.skill_cleaned)
print(sentence_embeddings2.shape)
print(sentence_embeddings2)

(289, 768)
[[-3.63384992e-01 -8.92237993e-04  1.66631150e+00 ... -2.80820787e-01
  -2.11184658e-02  2.73724347e-01]
 [-5.14828265e-01  1.20649576e-01  1.51134276e+00 ... -4.22038496e-01
  -5.03808483e-02  3.07444006e-01]
 [-2.85671383e-01  6.51391327e-01  1.55751550e+00 ... -6.51182830e-01
  -1.78134009e-01  4.17790771e-01]
 ...
 [-1.37683153e-01  3.98263693e-01  1.84874272e+00 ... -5.89098334e-01
   2.76534289e-01  3.61759186e-01]
 [-3.06679010e-02  3.77985090e-01  1.49822819e+00 ... -5.91389298e-01
  -1.04484595e-01 -3.78440231e-01]
 [-3.30874413e-01 -1.29140690e-01  1.29375446e+00 ... -4.28676575e-01
  -4.34687525e-01  2.39103541e-01]]


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
len(sentence_embeddings)

120

In [16]:
deni = []

for i in range (len(sentence_embeddings)):
   den= cosine_similarity([sentence_embeddings[i]],sentence_embeddings2)
   deni.append(den)
   print(deni)

Output hidden; open in https://colab.research.google.com to view.

In [18]:
import heapq

tfi = []
for i in range (len(deni)):
  for j in range (len(deni[i])):
    g=heapq.nlargest(5,range(len(deni[i][j])),np.array(deni[i][j]).__getitem__)
    tfi.append(g)
#g=heapq.nlargest(5,range(len(deni[0][0])),np.array(deni[0][0]).__getitem__)
#len(np.array(deni[]))
tfi


[[57, 74, 56, 76, 219],
 [22, 286, 141, 166, 73],
 [286, 264, 22, 281, 102],
 [286, 22, 256, 198, 123],
 [286, 281, 8, 102, 198],
 [10, 22, 286, 215, 102],
 [22, 141, 286, 166, 8],
 [283, 10, 22, 286, 183],
 [286, 102, 8, 109, 117],
 [22, 286, 198, 256, 123],
 [22, 17, 123, 286, 232],
 [286, 102, 8, 109, 264],
 [22, 232, 256, 238, 286],
 [166, 92, 22, 237, 50],
 [22, 198, 73, 8, 215],
 [22, 286, 8, 141, 17],
 [183, 22, 141, 237, 166],
 [22, 232, 123, 286, 17],
 [22, 141, 166, 183, 238],
 [96, 50, 82, 183, 286],
 [22, 141, 198, 123, 286],
 [286, 8, 102, 109, 123],
 [74, 75, 100, 57, 233],
 [22, 141, 183, 73, 166],
 [286, 22, 141, 256, 198],
 [286, 22, 264, 123, 198],
 [109, 198, 102, 264, 286],
 [22, 17, 141, 102, 123],
 [286, 198, 22, 8, 256],
 [22, 286, 198, 141, 73],
 [166, 237, 224, 47, 183],
 [286, 8, 102, 198, 215],
 [166, 237, 259, 22, 10],
 [286, 22, 232, 256, 141],
 [286, 102, 22, 8, 264],
 [270, 286, 114, 22, 263],
 [286, 264, 102, 22, 8],
 [10, 166, 22, 183, 198],
 [215, 102,

In [19]:
tfi= np.array(tfi)
tfi.shape

(120, 5)

In [98]:
p = []
for i in range (len(tfi)):
  k = df2.iloc[tfi[i]]
  p.append(k)
#print(p)
#p = pd.DataFrame(p)
df3 = df.assign(skill=p)
df3
df3.to_csv('file.csv')
files.download('file.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>